In [28]:
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#Carregar o modelo normalizador
from pickle import load
normalizador_iris = load(open(
    '/content/drive/MyDrive/Colab Notebooks/2025/models/normalizador_iris.model',
    'rb'))

#Carregar o modelo de clusters
iris_cluster_model = load(open(
    '/content/drive/MyDrive/Colab Notebooks/2025/models/iris_clusters.model',
    'rb'))


In [62]:
#Capturar os dados de uma nova instância
flor_num = [6.4, 2.8, 5.6, 2.1]
flor_cat = ['iris_setosa'] #este dado vai causar stress

#normalizar os dados numéricos de entrada
flor_num = normalizador_iris.transform([flor_num])
flor_num



/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[0.58333333, 0.33333333, 0.77966102, 0.83333333]])

In [63]:
#Estruturar os dados da novo flor
import pandas as pd
Colunas = ['sepal_length',
           'sepal_width',
           'petal_length',
           'petal_width',
           'Iris-setosa',
           'Iris-versicolor',
           'Iris-virginica']

nova_flor_data_frame = pd.DataFrame(columns = Colunas)
nova_flor_data_frame['sepal_length']=[flor_num[0][0]]
nova_flor_data_frame['sepal_width']=[flor_num[0][1]]
nova_flor_data_frame['petal_length']=[flor_num[0][2]]
nova_flor_data_frame['petal_width']=[flor_num[0][3]]
nova_flor_data_frame['Iris-setosa']=[0]
nova_flor_data_frame['Iris-versicolor']=[0]
nova_flor_data_frame['Iris-virginica']=[0]
nova_flor_data_frame

,sepal_length,sepal_width,petal_length,petal_width,Iris-setosa,Iris-versicolor,Iris-virginica
0,0.583333,0.333333,0.779661,0.833333,0,0,0


In [64]:
#Inferir o grupo ao qual essa nova flor pertence
grupo = iris_cluster_model.predict(nova_flor_data_frame)

cluster_nova_flor = iris_cluster_model.cluster_centers_[grupo[0]]
#Converter o centroid em um dataframe com a estrutura dos dados de treinamento
cluster_nova_flor = pd.DataFrame([cluster_nova_flor], columns=Colunas)

#Desnormalizar os dados numéricos
cluster_nova_flor_num = normalizador_iris.inverse_transform(cluster_nova_flor.drop(columns=['Iris-setosa', 'Iris-versicolor',	'Iris-virginica']))
#Converter os dados numericos desnormalizados em Data Frame
#Atenção: somente as colunas numéricas
cluster_nova_flor_num = pd.DataFrame(cluster_nova_flor_num, columns=['sepal_length','sepal_width','petal_length','petal_width'])
iris_cluster_model.cluster_centers_[grupo[0]]

array([6.31944444e-01, 4.68750000e-01, 7.66101695e-01, 8.75000000e-01,
       5.55111512e-17, 5.55111512e-17, 1.00000000e+00])

In [65]:
#Desnormalizar os dados categóricos
cluster_nova_flor_cat = round(cluster_nova_flor[['Iris-setosa','Iris-versicolor','Iris-virginica']],0)
cluster_nova_flor_cat
cluster_nova_flor_cat_desnormalizado = pd.from_dummies(cluster_nova_flor_cat, sep ='-' )
cluster_nova_flor_cat_desnormalizado

,Iris
0,virginica


In [66]:
#Compor o dataframe completo com os dados do centroide
centroide = cluster_nova_flor_num.join(cluster_nova_flor_cat_desnormalizado)
centroide

,sepal_length,sepal_width,petal_length,petal_width,Iris
0,6.575,3.125,5.52,2.2,virginica
